In [1]:
import requests
from pymongo import MongoClient
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import urllib2
import sys

## To write down all the urls to be scraped for articles and comments

In [9]:
# url = "https://www.theguardian.com/us/commentisfree"

# Which links to look for.


-
get_urls()

Total number of links stored is  20


## To download comments from guardian website

In [10]:
def download_comments(guardianLink):
    '''
    This function downloads the comments, along with title, topic of the original thread
    comment_id, no_upvotes, no_replies, comment_author, comment_author_id, etc
    Returns BeautifulSoup; need more extraction from the soup for the above-mentioned 
    parameters
    
    '''
        
    #Figuring out the PageId
    pageResponse = urllib2.urlopen(guardianLink)
    commentsMatch = re.search(r'/p/(.*?)"', pageResponse.read(), re.M|re.I)
    # print "Comments Match = ", commentsMatch()
    if commentsMatch:
        pageId = commentsMatch.group(1)
        print '[+] pageId has been retrieved ('+pageId+')'
    else:
        sys.exit('[-] Could not retrieve pageId!')

    #Retrieving comments
    downloadCount = 1
    downloadError = 0

    text = ''

    while downloadError == 0:
        try:
            response = urllib2.urlopen('http://www.theguardian.com/discussion/p/' + pageId 
                                       + '?page=' + str(downloadCount))
            html = response.read()
            length = len(text)
            text += html
            downloadCount = downloadCount + 1

        except:
            downloadError = 1

    return BeautifulSoup(text, 'html.parser')

## To extract data from soup

#### Returns a list of lists - comment_od, comment_text, author_id, author_name, number_of_upvotes

In [11]:
def getting_comment_data(soup):
    '''
    This function takes in the soup of comments. Returns a list of list. 
    Each element in the list is a list of -
    comment_id, comment, author_id, author, number_of_upvotes
    
    '''
       
    auth_name_lst = []
    comm_id_lst = []
    auth_id_lst = []
    for lis in soup.find_all('li'):
        if 'data-comment-author-id' in (lis.attrs) and 'data-comment-id' in (lis.attrs)and 'data-comment-author' in (lis.attrs):
            auth_name_lst.append(lis.attrs['data-comment-author'].encode('utf-8').replace("  ", " "))
            auth_id_lst.append(int(lis.attrs['data-comment-author-id']))
            comm_id_lst.append(int(lis.attrs['data-comment-id']))

    comments_text = soup.findAll("div", { "class" : "d-comment__body" })
    recommends = soup.findAll("span", {"class" : "d-comment__recommend-count--old"})
    users = soup.findAll("span", {"itemprop" : "givenName"})

    comment_data_list = []
    comment_text_list = []
    comment_id_list = []
    author_id_list = []
    author_name_list = []
    upvotes_count_list = []

    i = 0
    j = 0
    for comment_text, upvotes, user, auth_name, auth_id, comment_id in zip(comments_text, recommends, users, auth_name_lst, auth_id_lst, comm_id_lst):
        i += 1
        if 'comment was removed by a moderator ' not in comment_text.text:
            j += 1
#             This is the count of comments not removed by the moderator
                    

#             if auth_name.strip() != user.text.encode('utf-8').strip():
#                 print "something is broken for -"+ auth_name.strip()+ "-"+ user.text.encode('utf-8').strip()
#             else :
            if not upvotes.txt:
                upvote = 0
            else : 
                upvote = int(upvotes.txt)
            comment_text_list.append(comment_text.text)
            comment_id_list.append(comment_id)
            author_id_list.append(auth_id)
            author_name_list.append(auth_name)
            upvotes_count_list.append(upvote)            

    return comment_text_list, comment_id_list, author_id_list, auth_name_lst, upvotes_count_list

In [12]:
url = "https://www.theguardian.com/commentisfree/2017/aug/13/america-carpet-bombed-north-korea-remember-that-past"
soup = download_comments(url)
getting_comment_data(soup)

[+] pageId has been retrieved (72e9y)


([u'\nSay what?And do you know how someone who wants something but is not willing to do the hard work is called anywhere in the world (outside maybe Trumpland)?\n',
  u'\nGood. I will be sure to repeat that sentence hourly from no one, lest I be mistaken for a North Korean double agent again...\n',
  u'\nYes.\n',
  u'\nAnd your original claim that a large percentage of comments here suggest that people don\'t think North Korea is "bad bad ba" is utterly laughable.\n',
  u"\n\nNorth Korea is very bad and South Korea is not.\n\nDon't know what you mean. Literally no idea. Nothing on this planet has ever been that simple. You sound like Donald Trump.\nNorth Korea is a terrifying, totalitarian hell-hole, led by a brutal family dynasty that maintains the status-quo purely for their own material gains, and will happily keep its populace in fear and poverty as long as they keep power, even if that means chucking nuclear weapons at innocent civilians. Happy now?\n",
  u'\nDallasWilliams: " ...

### To extract topic(s) and text of an article

In [13]:
def article_topics_title(url):
    '''
    Given a URL, this function returns topics and contents of the article
    
    '''

    soup = BeautifulSoup(requests.get(url).text, 'html.parser')

    topic_list = [topic.attrs['data-link-name'][9:] for topic in soup.findAll("a", {"class" : "submeta__link"})
        if 'data-link-name' in topic.attrs and 'keyword: ' in topic.attrs['data-link-name']]

    for title in soup.find("h1", {"class" : "content__headline"}):
        title = str(title.strip().encode('utf-8'))
        

    for tag in soup.find_all('svg'):
        tag.decompose()
    for tag in soup.find_all('figure'):
        tag.decompose()
    for tag in soup.find_all('aside'):
        tag.decompose()
    for tag in soup.find_all('span'):
        tag.decompose()
    
    article = [str(s) for s in soup.find("div", {"class" : "content__article-body from-content-api js-article__body"})]
    
  
    return topic_list, article, title

In [16]:
urls.count()

2

In [14]:
def getting_all_data():
    cursor = urls.find({})
    for document in cursor: 
        url = str(document['url'])
        id_n = document['_id']
        print "Working on this link", url

        soup = download_comments(url)
        #     comment_text_list, comment_id_list, author_id_list, auth_name_lst, upvotes_count_list
        comment_text_list, comment_id_list, author_id_list, auth_name_lst, upvotes_count_list = getting_comment_data(soup)

        topics_list, article, title = article_topics_title(url)
        print "Title of the article is: ", title

        urls.update_one({'url': url}, 
                        {"$set": {'title': title, 'article' : article, 'topics_list': topics_list,
                                 'comment_ids' : comment_id_list, 'comment_text' : comment_text_list,
                                 'author_ids' : author_id_list, 'author_name' : auth_name_lst,
                                 'upvotes' : upvotes_count_list}}, upsert=True)
        return
    
getting_all_data()

Working on this link https://www.theguardian.com/commentisfree/2017/jan/31/america-theresa-may-condemn-trump-travel-ban-controlling-borders
[+] pageId has been retrieved (5pxhq)
Title of the article is:  As a friend of America, Theresa May must condemn Trump’s travel ban


In [17]:
urls.count()

2

In [6]:
from pymongo import MongoClient

client = MongoClient()

db = client["guardian"] #This is the name of the database
urls = db["urls"] # this is the table in that database
from pprint import pprint
cursor = urls.find({})
i = 0
for document in cursor: 
    if i < 20:
        for k, v in document.iteritems():
            print k, v
    i += 1
#     pprint(document['upvotes'])


url https://www.theguardian.com/commentisfree/2017/jan/31/ed-miliband-former-labour-leader
_id 5a6b5d42817c91156099085a
id 9
url https://www.theguardian.com/commentisfree/2017/jan/31/labour-mps-duty-corbyn-whip-brexit
_id 5a6b5d42817c91156099085b
id 10
url https://www.theguardian.com/commentisfree/2017/jan/31/tory-moderates-must-rebel-rock-hard-brexit-pro-trump-alignment
_id 5a6b5d42817c91156099085c
id 11
url https://www.theguardian.com/commentisfree/2017/jan/31/partner-bisexual-lgbtq-sexual-health-healthcare
_id 5a6b5d42817c91156099085d
id 12
url https://www.theguardian.com/commentisfree/2017/jan/31/sally-yates-us-constitution-two-fatalities-monday-night-massacre
_id 5a6b5d42817c91156099085e
id 13
url https://www.theguardian.com/commentisfree/2017/jan/31/now-is-not-the-time-to-cut-penalty-rates-for-struggling-workers
_id 5a6b5d42817c91156099085f
id 14
url https://www.theguardian.com/commentisfree/2017/jan/31/in-a-global-trend-towards-crimmigration-australia-has-led-the-world
_id 5a6b5

In [ ]:
startTime = time.time()
stopTime = time.time()
runningTime = (stopTime - startTime)/60;
print '[=] ' + str(downloadCount-1) + ' pages have been downloaded in ' + str(round(runningTime,2)) + ' minutes'
